In [2]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, plot_importance, plot_tree

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [3]:
otu_path = r"D:\05_kuisu\project\15_MAI\Foxtail-millet-data-analysis\Figure2\OTU0.7.csv"
tslw_snp_path = r"D:\05_kuisu\project\15_MAI\Foxtail-millet-data-analysis\Figure2\TSLWSNP.csv"

df_otu = pd.read_csv(otu_path)
df_snp = pd.read_csv(tslw_snp_path)

In [4]:
df_otu.head()

,sample,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,OTU_192,...,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,Si_102,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,1.9282,...,-0.042467,0.012649,-0.008380,0.021947,-0.012261,-0.020675,-0.058306,0.013853,0.022379,-0.039418
1,Si_105,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,2.2394,...,0.022155,-0.005878,0.003327,-0.003975,0.032975,-0.080775,0.008381,0.093369,0.053368,0.076554
2,Si_106,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,2.6644,...,-0.036409,-0.007399,-0.009371,-0.020853,0.002437,-0.014775,-0.022514,0.016131,-0.010219,-0.000622
3,Si_108,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,3.5274,...,0.022437,-0.009861,0.005828,-0.003640,0.010850,-0.035365,-0.002141,0.017251,0.001907,0.031235
4,Si_109,3.3673,1.9244,1.2618,3.4310,3.5506,2.7207,0.95007,4.7600,1.9244,...,0.037184,-0.007725,0.008184,0.021918,0.023928,-0.047812,0.005075,0.053935,0.017500,0.025807


In [5]:
df_snp.head()

,sample,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,Si_109,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [25]:
tslw_idx = [0,1007]
tslw_idx.extend([i for i in range(1,1005)])
tslw_idx.extend([i for i in range(1014, 1024)])
df_tslw = pd.merge(df_otu.iloc[:,tslw_idx], df_snp, on="sample")
df_tslw.head()

,sample,TSLW,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,...,PC10,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,3.20,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,...,-0.039418,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,4.07,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,...,0.076554,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,3.43,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,...,-0.000622,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,3.23,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,...,0.031235,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,Si_109,3.53,3.3673,1.9244,1.2618,3.4310,3.5506,2.7207,0.95007,4.7600,...,0.025807,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [87]:
data = df_tslw.dropna(axis=0,how='any')
print(data.shape)
data.head()

(680, 1025)


,sample,TSLW,OTU_97,OTU_17917,OTU_164,OTU_110,OTU_28612,OTU_42,OTU_281,OTU_28306,...,PC10,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9
0,Si_102,3.20,3.4726,1.4241,1.4241,1.9282,1.4241,2.8425,1.42410,5.1616,...,-0.039418,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,Si_105,4.07,3.7887,1.7997,1.1639,3.2757,5.0735,3.1469,1.61720,5.2247,...,0.076554,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,Si_106,3.43,3.2836,2.1369,1.9681,2.6644,3.2836,3.4207,0.97884,5.6724,...,-0.000622,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Si_108,3.23,4.1436,2.9225,2.4297,2.8365,4.9466,2.9225,0.90887,5.4595,...,0.031235,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,Si_113,2.97,2.7158,1.0091,1.0091,2.4743,3.1861,3.1035,1.59710,4.9002,...,0.015700,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [88]:
data["TSLW"].values

array([3.2 , 4.07, 3.43, 3.23, 2.97, 2.13, 2.17, 2.4 , 2.4 , 2.93, 1.87,
       3.03, 2.77, 2.83, 2.63, 2.97, 2.9 , 2.67, 2.77, 2.93, 2.5 , 2.57,
       2.27, 2.87, 2.73, 3.47, 2.53, 2.93, 2.73, 2.87, 3.17, 2.5 , 2.6 ,
       2.77, 2.53, 2.6 , 2.67, 2.93, 2.87, 3.13, 3.3 , 2.8 , 3.03, 2.87,
       3.07, 3.47, 3.  , 2.9 , 3.13, 3.13, 3.07, 2.87, 3.3 , 3.6 , 2.83,
       2.87, 2.97, 2.9 , 2.7 , 2.77, 2.2 , 2.63, 2.7 , 2.67, 2.3 , 2.6 ,
       2.87, 2.77, 2.5 , 2.3 , 2.53, 2.57, 3.  , 2.63, 2.77, 2.5 , 2.27,
       2.8 , 2.97, 3.17, 2.73, 2.5 , 3.43, 3.13, 2.97, 2.97, 2.65, 3.23,
       2.87, 2.67, 2.77, 2.73, 2.97, 2.23, 2.57, 2.8 , 2.8 , 2.8 , 2.6 ,
       2.97, 2.97, 2.87, 2.53, 2.8 , 2.8 , 2.67, 3.07, 3.37, 3.03, 2.63,
       2.9 , 2.9 , 2.63, 3.17, 3.3 , 2.27, 3.5 , 3.03, 2.47, 2.47, 3.  ,
       2.53, 2.5 , 2.27, 2.3 , 1.9 , 2.57, 2.3 , 2.17, 2.73, 2.94, 2.33,
       2.5 , 2.43, 2.07, 3.1 , 2.97, 2.5 , 3.07, 2.63, 2.67, 2.47, 2.8 ,
       2.93, 3.33, 2.7 , 2.93, 3.  , 3.6 , 3.03, 3.

In [89]:
from sklearn.linear_model import LinearRegression
from scipy.stats import t

results = []
for i in range(2,1006):
    variable_name = data.columns[i]
    X = data.iloc[:,i].values.reshape(-1,1)
    Y = data["TSLW"].values
    model = LinearRegression()
    model.fit(X,Y)
    coef = model.coef_[0]
    predictions = model.predict(X)
    n = len(X)
    mse = np.mean((Y - predictions)**2)
    std_err = np.sqrt(mse * (np.linalg.inv(np.dot(X.T, X)))[0, 0])
    # 计算 t 值
    t_val = coef / std_err
    # 需要样本数和自由度来计算p值
    degrees_of_freedom = n - 2
    # 计算p值 (双尾)
    p_val = 2 * (1 - t.cdf(abs(t_val), df=degrees_of_freedom))
    # 添加结果到列表中
    results.append([variable_name, coef, std_err, p_val])
r = pd.DataFrame(results, columns=["V1", "Estimate","Std.Error", "P"])
r.head()

,V1,Estimate,Std.Error,P
0,OTU_97,0.004898,0.003979,2.188144e-01
1,OTU_17917,0.007185,0.007702,3.512216e-01
2,OTU_164,-0.000606,0.007938,9.391253e-01
3,OTU_110,0.015101,0.004851,1.930226e-03
4,OTU_28612,0.018342,0.003359,6.677754e-08


In [90]:
r_order = r.sort_values(by=["P"],ascending=False)
r_order.head()

,V1,Estimate,Std.Error,P
239,OTU_7084,-0.000032,0.006733,0.996152
189,OTU_59,-0.000065,0.004496,0.988408
95,OTU_320,0.000173,0.010584,0.986970
427,OTU_383,0.000139,0.008467,0.986904
452,OTU_2690,-0.000175,0.009088,0.984620


In [107]:
## 仅使用SNP进行预测
import statsmodels.api as sm
import re

# 获取所有列名为”snp“开头的列
cc = list(r_order["V1"][:200].values) # 相关性较高的top200
snp_columns = [col for col in data.columns if re.match(r"^snp", col)]
otu_columns = [col for col in data.columns if re.match(r"OTU", col)]
pc_columns = [col for col in data.columns if re.match(r"^PC", col)]
# 创建初始线性回归模型
X = data[snp_columns]
y = data['TSLW']
X = sm.add_constant(X) # 添加常数项
TSLWm1 = sm.OLS(y, X).fit()
print(TSLWm1.summary())

# 使用逐步回归（向后逐步回归）
def backward_elimination(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    while len(initial_features) > 0:
        X = sm.add_constant(data[initial_features])
        model = sm.OLS(target, X).fit()
        p_values = model.tvalues.iloc[1:]  # 不包括常数项的 p 值
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            initial_features.remove(excluded_feature)
        else:
            break
    return initial_features
selected_features = backward_elimination(data[snp_columns], y)
X_selected = sm.add_constant(data[selected_features])
TSLWm = sm.OLS(y, X_selected).fit()
print(f"TSLWm1: X_selected shape: {X_selected.shape}\n",TSLWm.summary())

# 更新模型，加入新的变量CC
X_selected = sm.add_constant(data[selected_features])
X_selected = pd.concat([X_selected, data[cc]], axis=1)
TSLWm2 = sm.OLS(y, X_selected).fit()
print(f"TSLWm2： X_selected shape: {X_selected.shape}\n",TSLWm2.summary())

## 再次进行向后逐步回归
selected_features = backward_elimination(X_selected, y)
X_selected = sm.add_constant(X_selected[selected_features])
TSLWm = sm.OLS(y, X_selected).fit()
print(TSLWm.summary())

# 创建snp和OTU交互并更新模型
# 提取 snp 和 OTU 变量名
name_snp = [var for var in selected_features if re.match(r"^snp", var)]
name_otu = [var for var in selected_features if re.match(r"OTU", var)]

interaction_terms = pd.DataFrame()
for snp in name_snp:
    for otu in name_otu:
        interaction_terms[f"{snp}:{otu}"] = data[snp] * data[otu]


X_selected = pd.concat([X_selected, interaction_terms], axis=1)
selected_features = backward_elimination(X_selected, y)
X_selected = sm.add_constant(X_selected[selected_features])
TSLWm3 = sm.OLS(y, X_selected).fit()
print(f"TSLWm3： X_selected shape: {X_selected.shape}\n",TSLWm3.summary())

# 再次进行向后逐步回归
selected_features = backward_elimination(X_selected, y)
X_selected = sm.add_constant(X_selected[selected_features])
TSLWm = sm.OLS(y, X_selected).fit()
print(TSLWm.summary())

# 添加 PC 变量并更新模型
X_selected = pd.concat([X_selected, data[pc_columns]], axis=1)
selected_features = backward_elimination(X_selected, y)
X_selected = sm.add_constant(X_selected[selected_features])
TSLWm4 = sm.OLS(y, X_selected).fit()
print(f"TSLWm4： X_selected shape: {X_selected.shape}\n",TSLWm4.summary())

# 再次进行向后逐步回归
selected_features = backward_elimination(X_selected, y)
X_selected = sm.add_constant(X_selected[selected_features])
TSLWm = sm.OLS(y, X_selected).fit()
print(TSLWm.summary())

                            OLS Regression Results                            
Dep. Variable:                   TSLW   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     11.73
Date:                Tue, 14 May 2024   Prob (F-statistic):           1.07e-15
Time:                        19:37:01   Log-Likelihood:                -176.27
No. Observations:                 680   AIC:                             370.5
Df Residuals:                     671   BIC:                             411.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.3221      0.449      5.172      0.0

,snp1,snp2,snp3,snp4,snp5,snp6,snp7,snp8,snp9,OTU_7084,...,OTU_617,OTU_14661,OTU_25514,OTU_23468,OTU_24877,OTU_5415,OTU_14036,OTU_3690,OTU_5328,OTU_20590
0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,1.6981,...,1.6981,0.00000,1.9282,1.4241,1.08570,0.00000,1.69810,2.1266,2.3011,2.1266
1,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.9616,...,1.4083,0.86958,4.9820,2.7634,2.36050,0.00000,1.16390,3.6048,2.6727,2.3605
2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.2965,...,1.7770,0.57089,1.7770,1.5567,0.57089,0.00000,0.57089,1.7770,2.6644,2.1369
3,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.7451,...,3.4132,0.90887,3.4132,1.6753,0.52485,0.52485,1.46210,2.1732,1.8611,3.1535
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.5971,...,3.4759,1.33280,2.3365,1.3328,1.00910,0.00000,1.59710,1.5971,2.9226,1.8203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.2451,...,4.8082,2.36630,3.4562,3.0843,0.87283,1.62210,1.41280,3.9966,2.8549,2.6788
822,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.4170,...,2.9407,1.22380,2.8094,4.1722,2.11730,2.41700,1.73870,2.5870,3.1176,4.1452
823,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6844,...,3.0670,2.22960,2.7874,1.6844,0.79743,0.79743,1.30800,3.5934,0.0000,1.3080
824,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.3049,...,0.0000,1.30490,0.0000,2.7829,1.30490,1.30490,1.97860,2.4361,1.3049,3.0623


In [59]:
from sklearn.feature_selection import RFE
# split
snp_columns = [col for col in data.columns if col.startswith('snp')]

# 准备特征和目标变量
X = data[snp_columns]
y = data['TSLW']

# 拆分数据集以进行训练和测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化线性回归模型
model1 = LinearRegression()

# 使用RFE进行特征选择
selector = RFE(model1, n_features_to_select=1, step=1)
selector = selector.fit(X_train, y_train)

# 打印特征选择的支持和排名
print("Support:", selector.support_)
print("Ranking:", selector.ranking_)

# 选择特征
X_train_selected = X_train[X_train.columns[selector.support_]]
X_test_selected = X_test[X_test.columns[selector.support_]]

# 拟合最终模型
model2 = LinearRegression()
model2.fit(X_train_selected, y_train)

# 打印模型的系数和截距
print("Coefficients:", model2.coef_)
print("Intercept:", model2.intercept_)

# 重新准备特征和目标变量，包含新特征
X_new = data[snp_columns + cc]
y_new = data['TSLW']

# 拆分新的训练和测试集
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# 重新拟合更新后的模型
model3 = LinearRegression()
model3.fit(X_train_new, y_train_new)

# 打印更新后模型的系数和截距
print("Updated Coefficients:", model3.coef_)
print("Updated Intercept:", model3.intercept_)

Support: [ True False False False False False False False False]
Ranking: [1 2 7 6 9 3 5 4 8]
Coefficients: [0.28137436]
Intercept: 2.6698467571644042
Updated Coefficients: [ 3.77634893e-01  2.43741909e-01 -6.63016104e-02 -2.68418287e+11
  2.68418287e+11  1.57867615e-01 -1.05939628e-01  1.05804976e-01
  3.68219916e-02 -2.10447311e-02  1.03416443e-02  4.91333008e-03
 -3.24249268e-04 -1.12438202e-03 -6.11305237e-03  1.51290894e-02
 -8.68034363e-03  1.28974915e-02  6.02722168e-03 -1.44548416e-02
  2.46772766e-02 -1.97582245e-02  2.75802612e-02 -2.21309662e-02
  1.21634007e-02  5.46050072e-03  4.19812202e-02 -1.57775879e-02
 -1.58023834e-02 -1.15966797e-03 -9.66072083e-03  1.66101456e-02
 -8.69083405e-03  2.93064117e-02  8.89587402e-03 -1.45354271e-02
  8.86154175e-03 -1.42426491e-02 -2.81825066e-02  1.33781433e-02
  2.15778351e-02  2.63824463e-02  1.68037415e-02 -1.61170959e-04
  5.56182861e-03  1.20637417e-02 -3.04527283e-02 -4.52041626e-03
 -1.37825012e-02 -6.18743896e-03 -2.35290527e-0

In [66]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, f

# 假设我们已经有训练好的模型和测试数据
predictions = model3.predict(X_test_new)

# 计算均方预测误差 (MSPE)
MSPE = mean_squared_error(y_test_new, predictions)

# 计算皮尔逊相关系数
PearsonE, PearsonP = pearsonr(y_test_new, predictions)

# 计算调整后的R平方
n = len(y_test_new)
p = len(model3.coef_)
Rsquare = model3.score(X_test_new, y_test_new)
print(f"MSPE: {MSPE}, Rsquare: {Rsquare}, PearsonE: {PearsonE}, PearsonP: PearsonP:{PearsonP}")

MSPE: 0.18594212076922859, Rsquare: -0.612479281648346, PearsonE: -0.13363458906111975, PearsonP: PearsonP:0.12089828159756066


In [84]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from scipy import stats

# 假设data_train1和data_test1是你的训练和测试数据集
# 例如:
# data_train1 = pd.DataFrame(...)  # 你的训练数据
# data_test1 = pd.DataFrame(...)   # 你的测试数据
data_train1, data_test1 = train_test_split(data, test_size=0.2, random_state=42)

# 第一步：初步回归模型
X_train = data_train1.drop(columns=['TSLW',"sample"])
y_train = data_train1['TSLW']
selected_features = cc

# 第二步：与name.snp和name.otu相关的交互项
name_snp = [col for col in selected_features if 'snp' in col]
name_otu = [col for col in selected_features if 'OTU' in col]

# 添加交互项
for snp in name_snp:
    for otu in name_otu:
        data_train1[snp + '*' + otu] = data_train1[snp] * data_train1[otu]
        data_test1[snp + '*' + otu] = data_test1[snp] * data_test1[otu]

# 重新选择特征
X_train = data_train1.drop(columns=['TSLW', "sample"])

# 第三步：添加PC相关的特征
pc_features = [col for col in data_train1.columns if col.startswith('PC')]
X_train = data_train1[cc + pc_features]

# 最终模型
final_model = LinearRegression()
final_model.fit(X_train, y_train)

# 预测和评估
X_test = data_test1[X_train.columns]
y_test = data_test1['TSLW']
y_pred = final_model.predict(X_test)

# 计算MSPE
mspe = mean_squared_error(y_test, y_pred)
print(f'MSPE: {mspe}')

# 计算Pearson相关系数
pearson_corr, pearson_p = stats.pearsonr(y_test, y_pred)
print(f'Pearson Correlation: {pearson_corr}')
print(f'Pearson P-value: {pearson_p}')

# 计算调整后的R^2
r2 = final_model.score(X_test, y_test)
n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print(f'Adjusted R^2: {adj_r2}')

# 计算F统计量和P值
f_stat = final_model.score(X_train, y_train) / (1 - final_model.score(X_train, y_train)) * (n - p - 1) / p
p_value = 1 - stats.f.cdf(f_stat, p, n - p - 1)
print(f'F-statistic: {f_stat}')
print(f'P-value: {p_value}')

MSPE: 0.1919043986999862
Pearson Correlation: 0.0021867587434522217
Pearson P-value: 0.979842465948152
Adjusted R^2: 3.9955308578034545
F-statistic: -0.15712766802246703
P-value: nan


In [86]:
X_test.shape

(136, 210)